In [1]:
%pylab inline

import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pulp import *
import pickle
import pandas as pd

In [3]:
aminoAcidIndex = 'ACEDGFIHKMLNQPSRTWVY'


In [4]:
#Assuming 2 degree.
def addLinearConstraintsPulp(numAA,prob,allVariables,offset=1):
    
    allSingleAAConstraints = []
    
    for i in range(numAA):
        #blank = np.zeros(int((numAA*20)+((numAA*20)*((numAA*20)+1)/2)))
        prob+=lpSum(allVariables[i*20+j+offset] for j in range(20)) == 1
    
myPosLabels=['1E','1F','1G','2A','2B','2C','2D','2E','2F','2G','3A','3B','3C','3D','3E','3F','3G','4A','4B','4C','4D','4E']

#Assuming 2 degree.
def addPairConstraintsPulp(numAA,prob,allVariables,offset=1):
    signs = [1,-1]        
    arrayOffset = numAA*20
    arrayDim = numAA*20
    #Expanded Vector1
    for i in range(numAA*20):
        ##################################################
        tempArray=[]
        #Expanded Vector2        
        for j in range(i,numAA*20):

            if(j%20==0 and j!=i): # For each set of 20, add another constraint
                
                tempArray.append((allVariables[i+offset],-1)) # AA constraint for single term
                prob += LpAffineExpression(each for each in tempArray) == 0
                tempArray=[]
                tempArray.append((allVariables[arrayOffset+offset],1))
            else:
                tempArray.append((allVariables[arrayOffset+offset],1))

        
            arrayOffset+=1
            
        tempArray.append((allVariables[i+offset],-1))   
        prob += LpAffineExpression(each for each in tempArray) == 0
        ##################################################
        
        
        #################################################
        # Second Set of constraints to assure second variable is stable
        #################################################0
        
        
        tempArray2=[]
        #print(allVariables[i+offset], "=============", i)
        
        for k in range(i+1): # k is the number of times to add pair terms
            
            if(k>1 and k%20==0): #every 20, add another constraint that at most one pair can be selected
                
                # Assert that feature occurs as singleton
                tempArray2.append((allVariables[i+offset],-1))  
                prob += LpAffineExpression(each for each in tempArray2) == 0
                tempArray2=[]
            

            tempArray2.append((allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],1))
            #These calculations help get the index of the next pair term 
            
        
            #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],end=" ")
 
        
        #When complete, top it off and add the last set of constraints
        tempArray2.append((allVariables[i+offset],-1))
        #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],k,end=" ")
        prob += LpAffineExpression(each for each in tempArray2) == 0
        
        
def getPolyExpansionLabels(inputVector):
    
    myLabels = []
    

    myDegree = 2
    myNewVector=[]
    
    myLabels.append('Offset')
    for idx,each in enumerate(inputVector):
        myNewVector.append(np.sqrt(2)*each)
        myLabels.append(myPosLabels[idx//20]+"_"+aminoAcidIndex[idx%20])
    
    for i in range(len(inputVector)):
        for j in range(i,len(inputVector)):
            
            myLabels.append(myPosLabels[i//20]+"_"+aminoAcidIndex[i%20]+"__"+
            myPosLabels[j//20]+"_"+aminoAcidIndex[j%20])
    
    return myLabels

def getK(dim,iteration):
    mySum = 0
    for i in range(1,iteration+1):
        mySum+= (dim-i)
    return mySum

In [5]:
#Define Dummy Variable Encoding
aminoAcidIndex = 'ACEDGFIHKMLNQPSRTWVY'

def getAAVector(letter):
    vector = np.zeros(20);
    vector[aminoAcidIndex.index(letter)]=1
    return  vector

def encodeWithDummyVariables(sequence):
    newArray=[]
    for aa in sequence:
        newArray.append(getAAVector(aa))
    return np.array(newArray).ravel()

In [6]:
def addLinearConstraint_Forbiden(prob,allVariables,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bim):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
        
    for idx,each in enumerate(puma):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mixMut):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
    
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<25:
            allowedArray[idx]=0
    
    disallowed = (~allowedArray.astype(bool)).astype(int)
    
    tempArray=[]
    for idx,i in enumerate(disallowed):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == 0
    
    return allowedArray

In [7]:
bim = 'GRPEIWIAQELRRIGDEFNAYY'
puma = 'GQWAREIGAQLRRMADDLNAQY'
xMut = ['']*22
xMut[3] = 'DEHIKLMNQV'
xMut[6] ='DFHILNVY'
xMut[11]='AGIRTV'
xMut[13]='ADFGHILNPRSTVY' #C
xMut[14]='AG'
xMut[18]='ADHILNPTV'
xMut[19]='AEKT'
xMut[21]='ADGHNPRSTY' #C

mMut = ['']*22
mMut[2] = 'AGPRSTW'
mMut[4]='ADFGHLPRSVY' #C
mMut[5]='DEHQ'
mMut[6]='AITV'
mMut[7]='AGISTV'
mMut[11]='AEGIKRTV'
mMut[17]='ADFHILNPSTVY'
mMut[18]='DEHKNQ'

fMut = ['']*22
fMut[3] = 'AEIKLPQTV'
fMut[7]='ADGSY' #C
fMut[9]='DFGHILNRSVY'# C
fMut[13]='AFGILPRSTV' #C
fMut[16]='DEHIKLMNQV'
fMut[17]='ADFHILNPSTVY'
fMut[21]='AFILPSTV'

mixMut = []
for i in range(22):
    mixMut.append(set(xMut[i]).union(set(mMut[i])).union(set(fMut[i])))

In [8]:
mixMut

[set(),
 set(),
 {'A', 'G', 'P', 'R', 'S', 'T', 'W'},
 {'A', 'D', 'E', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'T', 'V'},
 {'A', 'D', 'F', 'G', 'H', 'L', 'P', 'R', 'S', 'V', 'Y'},
 {'D', 'E', 'H', 'Q'},
 {'A', 'D', 'F', 'H', 'I', 'L', 'N', 'T', 'V', 'Y'},
 {'A', 'D', 'G', 'I', 'S', 'T', 'V', 'Y'},
 set(),
 {'D', 'F', 'G', 'H', 'I', 'L', 'N', 'R', 'S', 'V', 'Y'},
 set(),
 {'A', 'E', 'G', 'I', 'K', 'R', 'T', 'V'},
 set(),
 {'A', 'D', 'F', 'G', 'H', 'I', 'L', 'N', 'P', 'R', 'S', 'T', 'V', 'Y'},
 {'A', 'G'},
 set(),
 {'D', 'E', 'H', 'I', 'K', 'L', 'M', 'N', 'Q', 'V'},
 {'A', 'D', 'F', 'H', 'I', 'L', 'N', 'P', 'S', 'T', 'V', 'Y'},
 {'A', 'D', 'E', 'H', 'I', 'K', 'L', 'N', 'P', 'Q', 'T', 'V'},
 {'A', 'E', 'K', 'T'},
 set(),
 {'A', 'D', 'F', 'G', 'H', 'I', 'L', 'N', 'P', 'R', 'S', 'T', 'V', 'Y'}]

In [9]:
designedLibSum = 0
for bg in [bim,puma]:

    for j in [xMut,fMut,mMut]:
        mixMutBG = []
        for i in range(22):
            if(len(j[i])==0):
                mixMutBG.append(set(bg[i]))
            else:
                mixMutBG.append(set(j[i]))

        thisCount = 1
        for i in mixMutBG:
            thisCount=thisCount*len(i)
        print(thisCount)
        designedLibSum+=thisCount
    
print(designedLibSum)

4838400
4752000
4257792
4838400
4752000
4257792
27696384


In [10]:
mixMutBG = []
for i in range(22):
    mixMutBG.append(set(xMut[i]).union(set(mMut[i])).union(set(fMut[i])).union(set(bim[i])).union(set(puma[i])))

In [11]:
mixMutBG

[{'G'},
 {'Q', 'R'},
 {'A', 'G', 'P', 'R', 'S', 'T', 'W'},
 {'A', 'D', 'E', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'T', 'V'},
 {'A', 'D', 'F', 'G', 'H', 'I', 'L', 'P', 'R', 'S', 'V', 'Y'},
 {'D', 'E', 'H', 'Q', 'W'},
 {'A', 'D', 'F', 'H', 'I', 'L', 'N', 'T', 'V', 'Y'},
 {'A', 'D', 'G', 'I', 'S', 'T', 'V', 'Y'},
 {'A', 'Q'},
 {'D', 'E', 'F', 'G', 'H', 'I', 'L', 'N', 'Q', 'R', 'S', 'V', 'Y'},
 {'L'},
 {'A', 'E', 'G', 'I', 'K', 'R', 'T', 'V'},
 {'R'},
 {'A', 'D', 'F', 'G', 'H', 'I', 'L', 'M', 'N', 'P', 'R', 'S', 'T', 'V', 'Y'},
 {'A', 'G'},
 {'D'},
 {'D', 'E', 'H', 'I', 'K', 'L', 'M', 'N', 'Q', 'V'},
 {'A', 'D', 'F', 'H', 'I', 'L', 'N', 'P', 'S', 'T', 'V', 'Y'},
 {'A', 'D', 'E', 'H', 'I', 'K', 'L', 'N', 'P', 'Q', 'T', 'V'},
 {'A', 'E', 'K', 'T'},
 {'Q', 'Y'},
 {'A', 'D', 'F', 'G', 'H', 'I', 'L', 'N', 'P', 'R', 'S', 'T', 'V', 'Y'}]

In [9]:
thisCount = 1
for i in mixMutBG:
    thisCount=thisCount*len(i)
print(thisCount)

879179857920000


In [10]:
libVariables = [LpVariable("i_"+j,0,22,LpInteger) for j in ['bx','bm','bf','px','pm','pf']] 
libVariablesBinary = [LpVariable("b_"+j,0,22,LpBinary) for j in ['bx','bm','bf','px','pm','pf']] 

def addLinearConstraintSingleLib(prob,allVariables,bgLib,mutAllowed,libVar,libVarBin,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bgLib):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mutAllowed):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
            
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<10:
            allowedArray[idx]=0
    
    tempArray=[]
    for idx,i in enumerate(allowedArray):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == libVar
    prob += libVar >= libVarBin*22 # libVarBin is 1 if libVar is >= 22
    #Because of the other constraints - this is just a variable that assigns boolean == 22
    
    return tempArray
    
def addLinearConstraintMultipleLib(prob,allVariables,offset=1):
    addLinearConstraintSingleLib(prob,allVariables,bim,xMut,libVariables[0],libVariablesBinary[0],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,mMut,libVariables[1],libVariablesBinary[1],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,fMut,libVariables[2],libVariablesBinary[2],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,xMut,libVariables[3],libVariablesBinary[3],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,mMut,libVariables[4],libVariablesBinary[4],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,fMut,libVariables[5],libVariablesBinary[5],offset)
def addLinearConstraint_library(prob,allVariables,offset=1):
    prob+= lpSum(each for each in libVariablesBinary) >= 1
    addLinearConstraintMultipleLib(prob,allVariables,offset)
    #print(libVariables)

# END OF GENERIC TEMPLATE

In [11]:
allData,allDataName = pickle.load(open('/home/vxue/data/sort_specificity/ncv_y/allData.pickle','rb'))

In [13]:
thisData = allData[allDataName.index('all_s')]
aaCountIndex = [i+"_"+j for i in myPosLabels for j in aminoAcidIndex]
thisData['aaEncoding'] = thisData.apply(lambda x: list(encodeWithDummyVariables(x.protein)),axis=1)
myMatrix = np.array([np.array(i) for i in thisData.aaEncoding])
aaCounts = myMatrix.sum(axis=0)

polyModel = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpolyall_x.pickle",'rb'))
weights = pd.read_pickle("/scratch/users/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpolyall_x.weights")

# For Affinity - Sanity Check

In [11]:
featureTable = getPolyExpansionLabels(np.ones(440))
allVariables=[LpVariable("x_"+j,0,1,LpBinary) for j in featureTable]

# defines the problem
prob = LpProblem("problem", LpMinimize)
addLinearConstraint_Forbiden(prob,allVariables)
#addLinearConstraint_library(prob,allVariables)
addLinearConstraintsPulp(22,prob,allVariables)
addPairConstraintsPulp(22,prob,allVariables)

prob += LpAffineExpression((allVariables[k],weights[k]) for k in range(len(allVariables)))
prob.solve(GLPK(msg=0))

1

In [12]:
allValues = [(idx,i,value(i)) for idx,i in enumerate(allVariables[1:]) if value(i)>0]
print("".join([str(allValues[i][1])[5:] for i in range(0,22)]))

GQWLIWFAAELRRLADEFNAQY


In [13]:
polyModel.predict(encodeWithDummyVariables('GQWLIWFAAELRRLADEFNAQY').reshape(1,-1))

array([-13.58956091])

# For Specificity

In [14]:
thisData_1 = allData[allDataName.index('all_n')]
thisData_2 = allData[allDataName.index('all_x')]

In [15]:
thisData_1['aaEncoding'] = thisData_1.apply(lambda x: list(encodeWithDummyVariables(x.protein)),axis=1)
myMatrix = np.array([np.array(i) for i in thisData.aaEncoding])
aaCounts1 = myMatrix.sum(axis=0)

thisData_2['aaEncoding'] = thisData_2.apply(lambda x: list(encodeWithDummyVariables(x.protein)),axis=1)
myMatrix = np.array([np.array(i) for i in thisData.aaEncoding])
aaCounts2 = myMatrix.sum(axis=0)

aaCounts = max(aaCounts1.tolist(),aaCounts2.tolist())

In [16]:
weights1 = pd.read_pickle("/scratch/users/vxue/data/sort_specificity/design/weights/ncv_z.modelpolyall_n.weights")
weights2 = pd.read_pickle("/scratch/users/vxue/data/sort_specificity/design/weights/ncv_z.modelpolyall_x.weights")
weights =  weights1 - weights2

In [17]:
polyModel_1 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_z/modelpolyall_n.pickle",'rb'))
polyModel_2 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_z/modelpolyall_x.pickle",'rb'))


In [18]:
featureTable = getPolyExpansionLabels(np.ones(440))
allVariables=[LpVariable("x_"+j,0,1,LpBinary) for j in featureTable]

# defines the problem
prob = LpProblem("problem", LpMinimize)
addLinearConstraint_Forbiden(prob,allVariables)
#addLinearConstraint_library(prob,allVariables)
addLinearConstraintsPulp(22,prob,allVariables)
addPairConstraintsPulp(22,prob,allVariables)

prob += LpAffineExpression((allVariables[k],weights[k]) for k in range(len(allVariables)))
prob.solve(GLPK(msg=0))

1

In [19]:
allValues = [(idx,i,value(i)) for idx,i in enumerate(allVariables[1:]) if value(i)>0]
print("".join([str(allValues[i][1])[5:] for i in range(0,22)]))

GRWDHEIAQELRRIGDEIDAYY


In [20]:
optSeq = "".join([str(allValues[i][1])[5:] for i in range(0,22)])


In [21]:
print(polyModel_1.predict(encodeWithDummyVariables(optSeq).reshape(1,-1)))
print(polyModel_2.predict(encodeWithDummyVariables(optSeq).reshape(1,-1)))

[-15.90610367]
[-9.39554194]


# Do it for all models 

# 1 vs 2

In [1]:
def kcalToKD(i):
    return np.e**(i/ ((1.9872041 * 10**-3)  * (298))) * 10**9

In [2]:
import numpy as np

In [7]:
kcalToKD(-11.5)

3.6829664604474637

In [3]:
kcalToKD(-9)

250.97648501805526

In [19]:
kcalToKD(-8.5)

583.8752770357092

In [9]:
kcalToKD(-9.7)

76.96108067660565

In [10]:
kcalToKD(-8.6)

493.1541312093924

In [5]:
kcalToKD(-9.5)-kcalToKD(-10.5)

87.94828873433416

In [4]:
kcalToKD(-9.5)

107.88125222874567

In [32]:
kcalToKD(-9.9)

54.90303288612328

In [29]:
%%writefile /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_1v2.py


from pulp import *
import pickle
import pandas as pd
import sys
import numpy as np

aminoAcidIndex = 'ACEDGFIHKMLNQPSRTWVY'
myPosLabels=['1E','1F','1G','2A','2B','2C','2D','2E','2F','2G','3A','3B','3C','3D','3E','3F','3G','4A','4B','4C','4D','4E']


def kcalToKD(i):
    return np.e**(i/ ((1.9872041 * 10**-3)  * (298))) * 10**9

###############################
# Define ILP Functions to generate constraints
###############################

#Assuming 2 degree.
def addLinearConstraintsPulp(numAA,prob,allVariables,offset=1):
    
    allSingleAAConstraints = []
    
    for i in range(numAA):
        #blank = np.zeros(int((numAA*20)+((numAA*20)*((numAA*20)+1)/2)))
        prob+=lpSum(allVariables[i*20+j+offset] for j in range(20)) == 1
    


#Assuming 2 degree.
def addPairConstraintsPulp(numAA,prob,allVariables,offset=1):
    signs = [1,-1]        
    arrayOffset = numAA*20
    arrayDim = numAA*20
    #Expanded Vector1
    for i in range(numAA*20):
        ##################################################
        tempArray=[]
        #Expanded Vector2        
        for j in range(i,numAA*20):

            if(j%20==0 and j!=i): # For each set of 20, add another constraint
                
                tempArray.append((allVariables[i+offset],-1)) # AA constraint for single term
                prob += LpAffineExpression(each for each in tempArray) == 0
                tempArray=[]
                tempArray.append((allVariables[arrayOffset+offset],1))
            else:
                tempArray.append((allVariables[arrayOffset+offset],1))

        
            arrayOffset+=1
            
        tempArray.append((allVariables[i+offset],-1))   
        prob += LpAffineExpression(each for each in tempArray) == 0
        ##################################################
        
        
        #################################################
        # Second Set of constraints to assure second variable is stable
        #################################################
        
        
        tempArray2=[]
        #print(allVariables[i+offset], "=============", i)
        
        for k in range(i+1): # k is the number of times to add pair terms
            
            if(k>1 and k%20==0): #every 20, add another constraint that at most one pair can be selected
                
                # Assert that feature occurs as singleton
                tempArray2.append((allVariables[i+offset],-1))  
                prob += LpAffineExpression(each for each in tempArray2) == 0
                tempArray2=[]
            

            tempArray2.append((allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],1))
            #These calculations help get the index of the next pair term 
            
        
            #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],end=" ")
 
        
        #When complete, top it off and add the last set of constraints
        tempArray2.append((allVariables[i+offset],-1))
        #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],k,end=" ")
        prob += LpAffineExpression(each for each in tempArray2) == 0
        
        
def getPolyExpansionLabels(inputVector):
    
    myLabels = []
    

    myDegree = 2
    myNewVector=[]
    
    myLabels.append('Offset')
    for idx,each in enumerate(inputVector):
        myNewVector.append(np.sqrt(2)*each)
        myLabels.append(myPosLabels[idx//20]+"_"+aminoAcidIndex[idx%20])
    
    for i in range(len(inputVector)):
        for j in range(i,len(inputVector)):
            
            myLabels.append(myPosLabels[i//20]+"_"+aminoAcidIndex[i%20]+"__"+
            myPosLabels[j//20]+"_"+aminoAcidIndex[j%20])
    
    return myLabels

def getK(dim,iteration):
    mySum = 0
    for i in range(1,iteration+1):
        mySum+= (dim-i)
    return mySum

###############################
# Define Dummy Variable Encoding
###############################

def getAAVector(letter):
    vector = np.zeros(20);
    vector[aminoAcidIndex.index(letter)]=1
    return  vector

def encodeWithDummyVariables(sequence):
    newArray=[]
    for aa in sequence:
        newArray.append(getAAVector(aa))
    return np.array(newArray).ravel()


def addLinearConstraint_Forbiden(prob,allVariables,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bim):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
        
    for idx,each in enumerate(puma):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mixMut):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
    
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<25:
            allowedArray[idx]=0
    
    disallowed = (~allowedArray.astype(bool)).astype(int)
    
    tempArray=[]
    for idx,i in enumerate(disallowed):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == 0
    
    return allowedArray

#############################
# Define Library Constraints
#############################

bim = 'GRPEIWIAQELRRIGDEFNAYY'
puma = 'GQWAREIGAQLRRMADDLNAQY'
xMut = ['']*22
xMut[3] = 'DEHIKLMNQV'
xMut[6] ='DFHILNVY'
xMut[11]='AGIRTV'
xMut[13]='ADFGHILNPRSTVY' #C
xMut[14]='AG'
xMut[18]='ADHILNPTV'
xMut[19]='AEKT'
xMut[21]='ADGHNPRSTY' #C

mMut = ['']*22
mMut[2] = 'AGPRSTW'
mMut[4]='ADFGHLPRSVY' #C
mMut[5]='DEHQ'
mMut[6]='AITV'
mMut[7]='AGISTV'
mMut[11]='AEGIKRTV'
mMut[17]='ADFHILNPSTVY'
mMut[18]='DEHKNQ'

fMut = ['']*22
fMut[3] = 'AEIKLPQTV'
fMut[7]='ADGSY' #C
fMut[9]='DFGHILNRSVY'# C
fMut[13]='AFGILPRSTV' #C
fMut[16]='DEHIKLMNQV'
fMut[17]='ADFHILNPSTVY'
fMut[21]='AFILPSTV'

mixMut = []
for i in range(22):
    mixMut.append(set(xMut[i]).union(set(mMut[i])).union(set(fMut[i])))

libVariables = [LpVariable("i_"+j,0,22,LpInteger) for j in ['bx','bm','bf','px','pm','pf']] 
libVariablesBinary = [LpVariable("b_"+j,0,22,LpBinary) for j in ['bx','bm','bf','px','pm','pf']] 



def addLinearConstraintSingleLib(prob,allVariables,bgLib,mutAllowed,libVar,libVarBin,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bgLib):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mutAllowed):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
            
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<10:
            allowedArray[idx]=0
    
    tempArray=[]
    for idx,i in enumerate(allowedArray):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == libVar
    prob += libVar >= libVarBin*22 # libVarBin is 1 if libVar is >= 22
    #Because of the other constraints - this is just a variable that assigns boolean == 22
    
    return tempArray
    
def addLinearConstraintMultipleLib(prob,allVariables,offset=1):
    addLinearConstraintSingleLib(prob,allVariables,bim,xMut,libVariables[0],libVariablesBinary[0],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,mMut,libVariables[1],libVariablesBinary[1],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,fMut,libVariables[2],libVariablesBinary[2],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,xMut,libVariables[3],libVariablesBinary[3],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,mMut,libVariables[4],libVariablesBinary[4],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,fMut,libVariables[5],libVariablesBinary[5],offset)
def addLinearConstraint_library(prob,allVariables,offset=1):
    prob+= lpSum(each for each in libVariablesBinary) >= 1
    addLinearConstraintMultipleLib(prob,allVariables,offset)
    #print(libVariables)

    
#############################
# Start loading model specific reqs
#############################

allData,allDataName = pickle.load(open('/home/vxue/data/sort_specificity/ncv_y/allData.pickle','rb'))

modelIndex = [allDataName.index(i) for i in ['all_x','all_s',
                                             'all_m','all_n',
                                             'all_f','all_t',
                                             'all_z','all_c']]

myIndex = [row+'_'+letter for row in myPosLabels for letter in aminoAcidIndex]

myCounts = dict()
for each,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myCounts[name]=posAA_toCount


designsTargets = [  #Do the pairs by the ones that have the best cross validated performance
    ('x','fn'),
    ('f','nx'),
    ('n','xf')]
    





slurmInput = int(sys.argv[1])    

cplexSolver = solvers.CPLEX_CMD("/scratch/users/mit_keating/Applications/CPLEX_Studio1271/cplex/bin/x86-64_linux/cplex")


for receptor1,receptor2 in [designsTargets[slurmInput]]:
    
    with open("/home/vxue/data/sort_specificity/design_y/specificity/trial1/"+receptor1+"-"+receptor2+"_poly.csv",'w') as outFile:

        model1 = "all_"+receptor1
        model2 = "all_"+receptor2[0]
        model3 = "all_"+receptor2[1]

        print(model1,model2,model3)
        aaCounts1  = [myCounts[model1][i] for i in myIndex]
        aaCounts2  = [myCounts[model2][i] for i in myIndex]
        aaCounts3  = [myCounts[model3][i] for i in myIndex]
        aaCounts = min(aaCounts1,aaCounts2,aaCounts3)


        featureTable = getPolyExpansionLabels(np.ones(440))
        allVariables=[LpVariable("x_"+j,0,1,LpBinary) for j in featureTable]

        polyModel1 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model1+".pickle",'rb'))
        weights1 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model1+".weights",'rb'))
        polyModel2 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model2+".pickle",'rb'))
        weights2 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model2+".weights",'rb'))
        polyModel3 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model3+".pickle",'rb'))
        weights3 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model3+".weights",'rb'))

        weights =  weights1 # Go for affinity with constraints on the affinity of the other 2
        weights[0] = 1 # Don't include the offset...



        # defines the problem
        prob = LpProblem("problem", LpMinimize)
        addLinearConstraint_Forbiden(prob,allVariables)
        addLinearConstraintsPulp(22,prob,allVariables)
        addPairConstraintsPulp(22,prob,allVariables)


        #-10.9085 | -8.5 were the original constriants - but it is hard to find examples that meet his for x-fn
        
        #-10.9085 | -9.5 Trial 1
        #-10.9085 | -9.0 Trial 2
        
        #Affinity for target must be tighter than 
        prob += LpAffineExpression((allVariables[k],weights1[k]) for k in range(len(allVariables))) <= -10.9085  -polyModel1.intercept_[0]
        #Affinity for offtarget must be less than 
        prob += LpAffineExpression((allVariables[k],weights2[k]) for k in range(len(allVariables))) >= -9.5 -polyModel2.intercept_[0]
        prob += LpAffineExpression((allVariables[k],weights3[k]) for k in range(len(allVariables))) >= -9.5 -polyModel3.intercept_[0]


        #Problem to Solve For
        prob += LpAffineExpression((allVariables[k],weights[k]) for k in range(len(allVariables)))


        topX = []
        for iteration in range(200):

            status = prob.solve(cplexSolver)
            LpStatus[status]

            if(LpStatus[status]=='Optimal'):
                
            
                allValues = [(idx,i,value(i)) for idx,i in enumerate(allVariables) if np.abs(value(i))>10**-6]
                optSeq = "".join([str(allValues[i][1])[5:] for i in range(0,22)])

                prob+= lpSum([allVariables[allValues[i][0]] for i in range(22)]) <= 21



                #optSeq = receptor1+"_"+receptor2+str(iteration)

                topX.append(optSeq)

                #optSeqScore1 = polyModel1.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                #optSeqScore2 = polyModel2.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                #optSeqScore3 = polyModel3.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]

                outFile.write(topX[-1])
                outFile.write("\n")
                outFile.flush()
            else:
                outFile.write(LpStatus[status])
                outFile.flush()
                break
                


            

Overwriting /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_1v2.py


In [2]:
%%writefile /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1.py


from pulp import *
import pickle
import pandas as pd
import sys
import numpy as np

aminoAcidIndex = 'ACEDGFIHKMLNQPSRTWVY'
myPosLabels=['1E','1F','1G','2A','2B','2C','2D','2E','2F','2G','3A','3B','3C','3D','3E','3F','3G','4A','4B','4C','4D','4E']


def kcalToKD(i):
    return np.e**(i/ ((1.9872041 * 10**-3)  * (298))) * 10**9

###############################
# Define ILP Functions to generate constraints
###############################

#Assuming 2 degree.
def addLinearConstraintsPulp(numAA,prob,allVariables,offset=1):
    
    allSingleAAConstraints = []
    
    for i in range(numAA):
        #blank = np.zeros(int((numAA*20)+((numAA*20)*((numAA*20)+1)/2)))
        prob+=lpSum(allVariables[i*20+j+offset] for j in range(20)) == 1
    


#Assuming 2 degree.
def addPairConstraintsPulp(numAA,prob,allVariables,offset=1):
    signs = [1,-1]        
    arrayOffset = numAA*20
    arrayDim = numAA*20
    #Expanded Vector1
    for i in range(numAA*20):
        ##################################################
        tempArray=[]
        #Expanded Vector2        
        for j in range(i,numAA*20):

            if(j%20==0 and j!=i): # For each set of 20, add another constraint
                
                tempArray.append((allVariables[i+offset],-1)) # AA constraint for single term
                prob += LpAffineExpression(each for each in tempArray) == 0
                tempArray=[]
                tempArray.append((allVariables[arrayOffset+offset],1))
            else:
                tempArray.append((allVariables[arrayOffset+offset],1))

        
            arrayOffset+=1
            
        tempArray.append((allVariables[i+offset],-1))   
        prob += LpAffineExpression(each for each in tempArray) == 0
        ##################################################
        
        
        #################################################
        # Second Set of constraints to assure second variable is stable
        #################################################
        
        
        tempArray2=[]
        #print(allVariables[i+offset], "=============", i)
        
        for k in range(i+1): # k is the number of times to add pair terms
            
            if(k>1 and k%20==0): #every 20, add another constraint that at most one pair can be selected
                
                # Assert that feature occurs as singleton
                tempArray2.append((allVariables[i+offset],-1))  
                prob += LpAffineExpression(each for each in tempArray2) == 0
                tempArray2=[]
            

            tempArray2.append((allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],1))
            #These calculations help get the index of the next pair term 
            
        
            #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],end=" ")
 
        
        #When complete, top it off and add the last set of constraints
        tempArray2.append((allVariables[i+offset],-1))
        #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],k,end=" ")
        prob += LpAffineExpression(each for each in tempArray2) == 0
        
        
def getPolyExpansionLabels(inputVector):
    
    myLabels = []
    

    myDegree = 2
    myNewVector=[]
    
    myLabels.append('Offset')
    for idx,each in enumerate(inputVector):
        myNewVector.append(np.sqrt(2)*each)
        myLabels.append(myPosLabels[idx//20]+"_"+aminoAcidIndex[idx%20])
    
    for i in range(len(inputVector)):
        for j in range(i,len(inputVector)):
            
            myLabels.append(myPosLabels[i//20]+"_"+aminoAcidIndex[i%20]+"__"+
            myPosLabels[j//20]+"_"+aminoAcidIndex[j%20])
    
    return myLabels

def getK(dim,iteration):
    mySum = 0
    for i in range(1,iteration+1):
        mySum+= (dim-i)
    return mySum

###############################
# Define Dummy Variable Encoding
###############################

def getAAVector(letter):
    vector = np.zeros(20);
    vector[aminoAcidIndex.index(letter)]=1
    return  vector

def encodeWithDummyVariables(sequence):
    newArray=[]
    for aa in sequence:
        newArray.append(getAAVector(aa))
    return np.array(newArray).ravel()


def addLinearConstraint_Forbiden(prob,allVariables,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bim):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
        
    for idx,each in enumerate(puma):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mixMut):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
    
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<25:
            allowedArray[idx]=0
    
    disallowed = (~allowedArray.astype(bool)).astype(int)
    
    tempArray=[]
    for idx,i in enumerate(disallowed):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == 0
    
    return allowedArray

#############################
# Define Library Constraints
#############################

bim = 'GRPEIWIAQELRRIGDEFNAYY'
puma = 'GQWAREIGAQLRRMADDLNAQY'
xMut = ['']*22
xMut[3] = 'DEHIKLMNQV'
xMut[6] ='DFHILNVY'
xMut[11]='AGIRTV'
xMut[13]='ADFGHILNPRSTVY' #C
xMut[14]='AG'
xMut[18]='ADHILNPTV'
xMut[19]='AEKT'
xMut[21]='ADGHNPRSTY' #C

mMut = ['']*22
mMut[2] = 'AGPRSTW'
mMut[4]='ADFGHLPRSVY' #C
mMut[5]='DEHQ'
mMut[6]='AITV'
mMut[7]='AGISTV'
mMut[11]='AEGIKRTV'
mMut[17]='ADFHILNPSTVY'
mMut[18]='DEHKNQ'

fMut = ['']*22
fMut[3] = 'AEIKLPQTV'
fMut[7]='ADGSY' #C
fMut[9]='DFGHILNRSVY'# C
fMut[13]='AFGILPRSTV' #C
fMut[16]='DEHIKLMNQV'
fMut[17]='ADFHILNPSTVY'
fMut[21]='AFILPSTV'

mixMut = []
for i in range(22):
    mixMut.append(set(xMut[i]).union(set(mMut[i])).union(set(fMut[i])))

libVariables = [LpVariable("i_"+j,0,22,LpInteger) for j in ['bx','bm','bf','px','pm','pf']] 
libVariablesBinary = [LpVariable("b_"+j,0,22,LpBinary) for j in ['bx','bm','bf','px','pm','pf']] 



def addLinearConstraintSingleLib(prob,allVariables,bgLib,mutAllowed,libVar,libVarBin,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bgLib):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mutAllowed):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
            
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<10:
            allowedArray[idx]=0
    
    tempArray=[]
    for idx,i in enumerate(allowedArray):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == libVar
    prob += libVar >= libVarBin*22 # libVarBin is 1 if libVar is >= 22
    #Because of the other constraints - this is just a variable that assigns boolean == 22
    
    return tempArray
    
def addLinearConstraintMultipleLib(prob,allVariables,offset=1):
    addLinearConstraintSingleLib(prob,allVariables,bim,xMut,libVariables[0],libVariablesBinary[0],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,mMut,libVariables[1],libVariablesBinary[1],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,fMut,libVariables[2],libVariablesBinary[2],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,xMut,libVariables[3],libVariablesBinary[3],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,mMut,libVariables[4],libVariablesBinary[4],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,fMut,libVariables[5],libVariablesBinary[5],offset)
def addLinearConstraint_library(prob,allVariables,offset=1):
    prob+= lpSum(each for each in libVariablesBinary) >= 1
    addLinearConstraintMultipleLib(prob,allVariables,offset)
    #print(libVariables)

    
#############################
# Start loading model specific reqs
#############################

allData,allDataName = pickle.load(open('/home/vxue/data/sort_specificity/ncv_y/allData.pickle','rb'))

modelIndex = [allDataName.index(i) for i in ['all_x','all_s',
                                             'all_m','all_n',
                                             'all_f','all_t',
                                             'all_z','all_c']]

myIndex = [row+'_'+letter for row in myPosLabels for letter in aminoAcidIndex]

myCounts = dict()
for each,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myCounts[name]=posAA_toCount


designsTargets = [  #Do the pairs by the ones that have the best cross validated performance
    ('fn','x'),
    ('nx','f'),
    ('xf','n')]





slurmInput = int(sys.argv[1])    


cplexSolver = solvers.CPLEX_CMD("/scratch/users/mit_keating/Applications/CPLEX_Studio1271/cplex/bin/x86-64_linux/cplex")

for receptor1,receptor2 in [designsTargets[slurmInput]]:
    
    with open("/home/vxue/data/sort_specificity/design_y/specificity/trial3/"+receptor1+"-"+receptor2+"_poly.csv",'w') as outFile:

        model1 = "all_"+receptor1[0]
        model2 = "all_"+receptor1[1]
        model3 = "all_"+receptor2


        print(model1,model2,model3)
        aaCounts1  = [myCounts[model1][i] for i in myIndex]
        aaCounts2  = [myCounts[model2][i] for i in myIndex]
        aaCounts3  = [myCounts[model3][i] for i in myIndex]
        aaCounts = min(aaCounts1,aaCounts2,aaCounts3)


        featureTable = getPolyExpansionLabels(np.ones(440))
        allVariables=[LpVariable("x_"+j,0,1,LpBinary) for j in featureTable]

        polyModel1 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model1+".pickle",'rb'))
        weights1 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model1+".weights",'rb'))
        polyModel2 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model2+".pickle",'rb'))
        weights2 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model2+".weights",'rb'))
        polyModel3 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model3+".pickle",'rb'))
        weights3 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model3+".weights",'rb'))

        weights =  weights3 # Go for the most destabilizing off target
        weights[0] = -1 # Don't include the offset...



        # defines the problem
        prob = LpProblem("problem", LpMaximize) #########<<<<<<<<<<<<<<<<<<<<< MAXIMIZE
        addLinearConstraint_Forbiden(prob,allVariables)
        addLinearConstraintsPulp(22,prob,allVariables)
        addPairConstraintsPulp(22,prob,allVariables)


        #Affinity for target must be tighter than 
        prob += LpAffineExpression((allVariables[k],weights1[k]) for k in range(len(allVariables))) <= -10 -polyModel1.intercept_[0]
        prob += LpAffineExpression((allVariables[k],weights2[k]) for k in range(len(allVariables))) <= -10 -polyModel2.intercept_[0]
        #Affinity for offtarget must be less than 
        prob += LpAffineExpression((allVariables[k],weights3[k]) for k in range(len(allVariables))) >= -8.5 -polyModel3.intercept_[0]


        #Problem to Solve For
        prob += LpAffineExpression((allVariables[k],weights[k]) for k in range(len(allVariables)))


        topX = []
        for iteration in range(1000):
            
            status = prob.solve(cplexSolver)
            LpStatus[status]
            
            if(LpStatus[status]=='Optimal'):                

                allValues = [(idx,i,value(i)) for idx,i in enumerate(allVariables) if np.abs(value(i))>10**-6]
                optSeq = "".join([str(allValues[i][1])[5:] for i in range(0,22)])

                prob+= lpSum([allVariables[allValues[i][0]] for i in range(22)]) <= 21



                #optSeq = receptor1+"_"+receptor2+str(iteration)

                topX.append(optSeq)

                #optSeqScore1 = polyModel1.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                #optSeqScore2 = polyModel2.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                #optSeqScore3 = polyModel3.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]

                outFile.write(topX[-1])
                outFile.write("\n")
                outFile.flush()
            else:
                outFile.write(LpStatus[status])
                outFile.flush()
                break


            

Overwriting /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1.py


# All scripts below now have constraints on the residues to only allow those that have been observed for the target binding

In [40]:
%%writefile /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1_bindingOnly.py


from pulp import *
import pickle
import pandas as pd
import sys
import numpy as np

aminoAcidIndex = 'ACEDGFIHKMLNQPSRTWVY'
myPosLabels=['1E','1F','1G','2A','2B','2C','2D','2E','2F','2G','3A','3B','3C','3D','3E','3F','3G','4A','4B','4C','4D','4E']


def kcalToKD(i):
    return np.e**(i/ ((1.9872041 * 10**-3)  * (298))) * 10**9

###############################
# Define ILP Functions to generate constraints
###############################

#Assuming 2 degree.
def addLinearConstraintsPulp(numAA,prob,allVariables,offset=1):
    
    allSingleAAConstraints = []
    
    for i in range(numAA):
        #blank = np.zeros(int((numAA*20)+((numAA*20)*((numAA*20)+1)/2)))
        prob+=lpSum(allVariables[i*20+j+offset] for j in range(20)) == 1
    


#Assuming 2 degree.
def addPairConstraintsPulp(numAA,prob,allVariables,offset=1):
    signs = [1,-1]        
    arrayOffset = numAA*20
    arrayDim = numAA*20
    #Expanded Vector1
    for i in range(numAA*20):
        ##################################################
        tempArray=[]
        #Expanded Vector2        
        for j in range(i,numAA*20):

            if(j%20==0 and j!=i): # For each set of 20, add another constraint
                
                tempArray.append((allVariables[i+offset],-1)) # AA constraint for single term
                prob += LpAffineExpression(each for each in tempArray) == 0
                tempArray=[]
                tempArray.append((allVariables[arrayOffset+offset],1))
            else:
                tempArray.append((allVariables[arrayOffset+offset],1))

        
            arrayOffset+=1
            
        tempArray.append((allVariables[i+offset],-1))   
        prob += LpAffineExpression(each for each in tempArray) == 0
        ##################################################
        
        
        #################################################
        # Second Set of constraints to assure second variable is stable
        #################################################
        
        
        tempArray2=[]
        #print(allVariables[i+offset], "=============", i)
        
        for k in range(i+1): # k is the number of times to add pair terms
            
            if(k>1 and k%20==0): #every 20, add another constraint that at most one pair can be selected
                
                # Assert that feature occurs as singleton
                tempArray2.append((allVariables[i+offset],-1))  
                prob += LpAffineExpression(each for each in tempArray2) == 0
                tempArray2=[]
            

            tempArray2.append((allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],1))
            #These calculations help get the index of the next pair term 
            
        
            #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],end=" ")
 
        
        #When complete, top it off and add the last set of constraints
        tempArray2.append((allVariables[i+offset],-1))
        #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],k,end=" ")
        prob += LpAffineExpression(each for each in tempArray2) == 0
        
        
def getPolyExpansionLabels(inputVector):
    
    myLabels = []
    

    myDegree = 2
    myNewVector=[]
    
    myLabels.append('Offset')
    for idx,each in enumerate(inputVector):
        myNewVector.append(np.sqrt(2)*each)
        myLabels.append(myPosLabels[idx//20]+"_"+aminoAcidIndex[idx%20])
    
    for i in range(len(inputVector)):
        for j in range(i,len(inputVector)):
            
            myLabels.append(myPosLabels[i//20]+"_"+aminoAcidIndex[i%20]+"__"+
            myPosLabels[j//20]+"_"+aminoAcidIndex[j%20])
    
    return myLabels

def getK(dim,iteration):
    mySum = 0
    for i in range(1,iteration+1):
        mySum+= (dim-i)
    return mySum

###############################
# Define Dummy Variable Encoding
###############################

def getAAVector(letter):
    vector = np.zeros(20);
    vector[aminoAcidIndex.index(letter)]=1
    return  vector

def encodeWithDummyVariables(sequence):
    newArray=[]
    for aa in sequence:
        newArray.append(getAAVector(aa))
    return np.array(newArray).ravel()


def addLinearConstraint_Forbiden(prob,allVariables,inputCounts,threshold,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bim):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
        
    for idx,each in enumerate(puma):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mixMut):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
    
    #Disallow low frequency observations
    for idx,each in enumerate(inputCounts):
        if(each)<threshold:
            allowedArray[idx]=0
    
    disallowed = (~allowedArray.astype(bool)).astype(int)
    
    tempArray=[]
    for idx,i in enumerate(disallowed):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == 0
    
    return allowedArray



#############################
# Define Library Constraints
#############################

bim = 'GRPEIWIAQELRRIGDEFNAYY'
puma = 'GQWAREIGAQLRRMADDLNAQY'
xMut = ['']*22
xMut[3] = 'DEHIKLMNQV'
xMut[6] ='DFHILNVY'
xMut[11]='AGIRTV'
xMut[13]='ADFGHILNPRSTVY' #C
xMut[14]='AG'
xMut[18]='ADHILNPTV'
xMut[19]='AEKT'
xMut[21]='ADGHNPRSTY' #C

mMut = ['']*22
mMut[2] = 'AGPRSTW'
mMut[4]='ADFGHLPRSVY' #C
mMut[5]='DEHQ'
mMut[6]='AITV'
mMut[7]='AGISTV'
mMut[11]='AEGIKRTV'
mMut[17]='ADFHILNPSTVY'
mMut[18]='DEHKNQ'

fMut = ['']*22
fMut[3] = 'AEIKLPQTV'
fMut[7]='ADGSY' #C
fMut[9]='DFGHILNRSVY'# C
fMut[13]='AFGILPRSTV' #C
fMut[16]='DEHIKLMNQV'
fMut[17]='ADFHILNPSTVY'
fMut[21]='AFILPSTV'

mixMut = []
for i in range(22):
    mixMut.append(set(xMut[i]).union(set(mMut[i])).union(set(fMut[i])))

libVariables = [LpVariable("i_"+j,0,22,LpInteger) for j in ['bx','bm','bf','px','pm','pf']] 
libVariablesBinary = [LpVariable("b_"+j,0,22,LpBinary) for j in ['bx','bm','bf','px','pm','pf']] 



def addLinearConstraintSingleLib(prob,allVariables,bgLib,mutAllowed,libVar,libVarBin,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bgLib):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mutAllowed):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
            
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<10:
            allowedArray[idx]=0
    
    tempArray=[]
    for idx,i in enumerate(allowedArray):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == libVar
    prob += libVar >= libVarBin*22 # libVarBin is 1 if libVar is >= 22
    #Because of the other constraints - this is just a variable that assigns boolean == 22
    
    return tempArray
    
def addLinearConstraintMultipleLib(prob,allVariables,offset=1):
    addLinearConstraintSingleLib(prob,allVariables,bim,xMut,libVariables[0],libVariablesBinary[0],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,mMut,libVariables[1],libVariablesBinary[1],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,fMut,libVariables[2],libVariablesBinary[2],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,xMut,libVariables[3],libVariablesBinary[3],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,mMut,libVariables[4],libVariablesBinary[4],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,fMut,libVariables[5],libVariablesBinary[5],offset)
def addLinearConstraint_library(prob,allVariables,offset=1):
    prob+= lpSum(each for each in libVariablesBinary) >= 1
    addLinearConstraintMultipleLib(prob,allVariables,offset)
    #print(libVariables)

    
#############################
# Start loading model specific reqs
#############################

allData,allDataName = pickle.load(open('/home/vxue/data/sort_specificity/ncv_y/allData.pickle','rb'))

modelIndex = [allDataName.index(i) for i in ['all_x','all_s',
                                             'all_m','all_n',
                                             'all_f','all_t',
                                             'all_z','all_c']]

myIndex = [row+'_'+letter for row in myPosLabels for letter in aminoAcidIndex]

myCounts = dict()
for each,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myCounts[name]=posAA_toCount
    
    
myBindingCounts = dict()
for each_full,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    #Count only the ones which have affinity less than ~50 nM
    each = each_full[each_full.yValue < -10.5].copy().reset_index()
    
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myBindingCounts[name]=posAA_toCount

    
designsTargets = [  #Do the pairs by the ones that have the best cross validated performance
    ('fn','x'),
    ('nx','f'),
    ('xf','n')]





slurmInput = int(sys.argv[1])    


cplexSolver = solvers.CPLEX_CMD("/scratch/users/mit_keating/Applications/CPLEX_Studio1271/cplex/bin/x86-64_linux/cplex")

for receptor1,receptor2 in [designsTargets[slurmInput]]:
    
    with open("/home/vxue/data/sort_specificity/design_y/specificity/trial8/"+receptor1+"-"+receptor2+"_poly.csv",'w') as outFile:

        model1 = "all_"+receptor1[0]
        model2 = "all_"+receptor1[1]
        model3 = "all_"+receptor2


        print(model1,model2,model3)
        aaCounts1  = [myCounts[model1][i] for i in myIndex]
        aaCounts2  = [myCounts[model2][i] for i in myIndex]
        aaCounts3  = [myCounts[model3][i] for i in myIndex]
        aaCounts = min(aaCounts1,aaCounts2,aaCounts3)


        #Restrict allowable sequences only to the ones that have observed the residue as a tight binder for the targets
        bindingCounts1 = [myBindingCounts[model1][i] for i in myIndex]
        bindingCounts2 = [myBindingCounts[model2][i] for i in myIndex]
        bindingCounts = min(bindingCounts1,bindingCounts2)
        
        
        
        featureTable = getPolyExpansionLabels(np.ones(440))
        allVariables=[LpVariable("x_"+j,0,1,LpBinary) for j in featureTable]

        polyModel1 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model1+".pickle",'rb'))
        weights1 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model1+".weights",'rb'))
        polyModel2 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model2+".pickle",'rb'))
        weights2 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model2+".weights",'rb'))
        polyModel3 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model3+".pickle",'rb'))
        weights3 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model3+".weights",'rb'))

        weights =  weights3 # Go for the most destabilizing off target
        weights[0] = -1 # Don't include the offset...



        # defines the problem
        prob = LpProblem("problem", LpMaximize) #########<<<<<<<<<<<<<<<<<<<<< MAXIMIZE
        addLinearConstraint_Forbiden(prob,allVariables,aaCounts,25)
        addLinearConstraint_Forbiden(prob,allVariables,bindingCounts,1)
        addLinearConstraintsPulp(22,prob,allVariables)
        addPairConstraintsPulp(22,prob,allVariables)


        #Affinity for target must be tighter than 
        prob += LpAffineExpression((allVariables[k],weights1[k]) for k in range(len(allVariables))) <= -10 -polyModel1.intercept_[0]
        prob += LpAffineExpression((allVariables[k],weights2[k]) for k in range(len(allVariables))) <= -10 -polyModel2.intercept_[0]
        #Affinity for offtarget must be less than 
        prob += LpAffineExpression((allVariables[k],weights3[k]) for k in range(len(allVariables))) >= -8.5 -polyModel3.intercept_[0]


        #Problem to Solve For
        prob += LpAffineExpression((allVariables[k],weights[k]) for k in range(len(allVariables)))


        topX = []
        for iteration in range(200):
            
            status = prob.solve(cplexSolver)
            LpStatus[status]
            
            if(LpStatus[status]=='Optimal'):                

                allValues = [(idx,i,value(i)) for idx,i in enumerate(allVariables) if np.abs(value(i))>10**-6]
                optSeq = "".join([str(allValues[i][1])[5:] for i in range(0,22)])

                prob+= lpSum([allVariables[allValues[i][0]] for i in range(22)]) <= 21



                #optSeq = receptor1+"_"+receptor2+str(iteration)

                topX.append(optSeq)

                #optSeqScore1 = polyModel1.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                #optSeqScore2 = polyModel2.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                #optSeqScore3 = polyModel3.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]

                outFile.write(topX[-1])
                outFile.write("\n")
                outFile.flush()
            else:
                outFile.write(LpStatus[status])
                outFile.flush()
                break


            

Overwriting /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1_bindingOnly.py


In [ ]:
#trial5
%%writefile /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1_edge.py


from pulp import *
import pickle
import pandas as pd
import sys
import numpy as np

aminoAcidIndex = 'ACEDGFIHKMLNQPSRTWVY'
myPosLabels=['1E','1F','1G','2A','2B','2C','2D','2E','2F','2G','3A','3B','3C','3D','3E','3F','3G','4A','4B','4C','4D','4E']


def kcalToKD(i):
    return np.e**(i/ ((1.9872041 * 10**-3)  * (298))) * 10**9

###############################
# Define ILP Functions to generate constraints
###############################

#Assuming 2 degree.
def addLinearConstraintsPulp(numAA,prob,allVariables,offset=1):
    
    allSingleAAConstraints = []
    
    for i in range(numAA):
        #blank = np.zeros(int((numAA*20)+((numAA*20)*((numAA*20)+1)/2)))
        prob+=lpSum(allVariables[i*20+j+offset] for j in range(20)) == 1
    


#Assuming 2 degree.
def addPairConstraintsPulp(numAA,prob,allVariables,offset=1):
    signs = [1,-1]        
    arrayOffset = numAA*20
    arrayDim = numAA*20
    #Expanded Vector1
    for i in range(numAA*20):
        ##################################################
        tempArray=[]
        #Expanded Vector2        
        for j in range(i,numAA*20):

            if(j%20==0 and j!=i): # For each set of 20, add another constraint
                
                tempArray.append((allVariables[i+offset],-1)) # AA constraint for single term
                prob += LpAffineExpression(each for each in tempArray) == 0
                tempArray=[]
                tempArray.append((allVariables[arrayOffset+offset],1))
            else:
                tempArray.append((allVariables[arrayOffset+offset],1))

        
            arrayOffset+=1
            
        tempArray.append((allVariables[i+offset],-1))   
        prob += LpAffineExpression(each for each in tempArray) == 0
        ##################################################
        
        
        #################################################
        # Second Set of constraints to assure second variable is stable
        #################################################
        
        
        tempArray2=[]
        #print(allVariables[i+offset], "=============", i)
        
        for k in range(i+1): # k is the number of times to add pair terms
            
            if(k>1 and k%20==0): #every 20, add another constraint that at most one pair can be selected
                
                # Assert that feature occurs as singleton
                tempArray2.append((allVariables[i+offset],-1))  
                prob += LpAffineExpression(each for each in tempArray2) == 0
                tempArray2=[]
            

            tempArray2.append((allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],1))
            #These calculations help get the index of the next pair term 
            
        
            #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],end=" ")
 
        
        #When complete, top it off and add the last set of constraints
        tempArray2.append((allVariables[i+offset],-1))
        #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],k,end=" ")
        prob += LpAffineExpression(each for each in tempArray2) == 0
        
        
def getPolyExpansionLabels(inputVector):
    
    myLabels = []
    

    myDegree = 2
    myNewVector=[]
    
    myLabels.append('Offset')
    for idx,each in enumerate(inputVector):
        myNewVector.append(np.sqrt(2)*each)
        myLabels.append(myPosLabels[idx//20]+"_"+aminoAcidIndex[idx%20])
    
    for i in range(len(inputVector)):
        for j in range(i,len(inputVector)):
            
            myLabels.append(myPosLabels[i//20]+"_"+aminoAcidIndex[i%20]+"__"+
            myPosLabels[j//20]+"_"+aminoAcidIndex[j%20])
    
    return myLabels

def getK(dim,iteration):
    mySum = 0
    for i in range(1,iteration+1):
        mySum+= (dim-i)
    return mySum

###############################
# Define Dummy Variable Encoding
###############################

def getAAVector(letter):
    vector = np.zeros(20);
    vector[aminoAcidIndex.index(letter)]=1
    return  vector

def encodeWithDummyVariables(sequence):
    newArray=[]
    for aa in sequence:
        newArray.append(getAAVector(aa))
    return np.array(newArray).ravel()


def addLinearConstraint_Forbiden(prob,allVariables,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bim):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
        
    for idx,each in enumerate(puma):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mixMut):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
    
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<25:
            allowedArray[idx]=0
    
    disallowed = (~allowedArray.astype(bool)).astype(int)
    
    tempArray=[]
    for idx,i in enumerate(disallowed):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == 0
    
    return allowedArray

#############################
# Define Library Constraints
#############################

bim = 'GRPEIWIAQELRRIGDEFNAYY'
puma = 'GQWAREIGAQLRRMADDLNAQY'
xMut = ['']*22
xMut[3] = 'DEHIKLMNQV'
xMut[6] ='DFHILNVY'
xMut[11]='AGIRTV'
xMut[13]='ADFGHILNPRSTVY' #C
xMut[14]='AG'
xMut[18]='ADHILNPTV'
xMut[19]='AEKT'
xMut[21]='ADGHNPRSTY' #C

mMut = ['']*22
mMut[2] = 'AGPRSTW'
mMut[4]='ADFGHLPRSVY' #C
mMut[5]='DEHQ'
mMut[6]='AITV'
mMut[7]='AGISTV'
mMut[11]='AEGIKRTV'
mMut[17]='ADFHILNPSTVY'
mMut[18]='DEHKNQ'

fMut = ['']*22
fMut[3] = 'AEIKLPQTV'
fMut[7]='ADGSY' #C
fMut[9]='DFGHILNRSVY'# C
fMut[13]='AFGILPRSTV' #C
fMut[16]='DEHIKLMNQV'
fMut[17]='ADFHILNPSTVY'
fMut[21]='AFILPSTV'

mixMut = []
for i in range(22):
    mixMut.append(set(xMut[i]).union(set(mMut[i])).union(set(fMut[i])))

libVariables = [LpVariable("i_"+j,0,22,LpInteger) for j in ['bx','bm','bf','px','pm','pf']] 
libVariablesBinary = [LpVariable("b_"+j,0,22,LpBinary) for j in ['bx','bm','bf','px','pm','pf']] 



def addLinearConstraintSingleLib(prob,allVariables,bgLib,mutAllowed,libVar,libVarBin,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bgLib):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mutAllowed):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
            
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<10:
            allowedArray[idx]=0
    
    tempArray=[]
    for idx,i in enumerate(allowedArray):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == libVar
    prob += libVar >= libVarBin*22 # libVarBin is 1 if libVar is >= 22
    #Because of the other constraints - this is just a variable that assigns boolean == 22
    
    return tempArray
    
def addLinearConstraintMultipleLib(prob,allVariables,offset=1):
    addLinearConstraintSingleLib(prob,allVariables,bim,xMut,libVariables[0],libVariablesBinary[0],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,mMut,libVariables[1],libVariablesBinary[1],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,fMut,libVariables[2],libVariablesBinary[2],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,xMut,libVariables[3],libVariablesBinary[3],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,mMut,libVariables[4],libVariablesBinary[4],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,fMut,libVariables[5],libVariablesBinary[5],offset)
def addLinearConstraint_library(prob,allVariables,offset=1):
    prob+= lpSum(each for each in libVariablesBinary) >= 1
    addLinearConstraintMultipleLib(prob,allVariables,offset)
    #print(libVariables)

    
#############################
# Start loading model specific reqs
#############################

allData,allDataName = pickle.load(open('/home/vxue/data/sort_specificity/ncv_y/allData.pickle','rb'))

modelIndex = [allDataName.index(i) for i in ['all_x','all_s',
                                             'all_m','all_n',
                                             'all_f','all_t',
                                             'all_z','all_c']]

myIndex = [row+'_'+letter for row in myPosLabels for letter in aminoAcidIndex]

myCounts = dict()
for each,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myCounts[name]=posAA_toCount


designsTargets = [  #Do the pairs by the ones that have the best cross validated performance
    ('fn','x'),
    ('nx','f'),
    ('xf','n')]

slurmInput =int(sys.argv[1])    


cplexSolver = solvers.CPLEX_CMD("/scratch/users/mit_keating/Applications/CPLEX_Studio1271/cplex/bin/x86-64_linux/cplex")

for receptor1,receptor2 in [designsTargets[slurmInput]]:
    
    with open("/home/vxue/data/sort_specificity/design_y/specificity/trial5/"+receptor1+"-"+receptor2+"_poly.csv",'w') as outFile:
        model1 = "all_"+receptor1[0]
        model2 = "all_"+receptor1[1]
        model3 = "all_"+receptor2


        print(model1,model2,model3)
        aaCounts1  = [myCounts[model1][i] for i in myIndex]
        aaCounts2  = [myCounts[model2][i] for i in myIndex]
        aaCounts3  = [myCounts[model3][i] for i in myIndex]
        aaCounts = min(aaCounts1,aaCounts2,aaCounts3)


        featureTable = getPolyExpansionLabels(np.ones(440))
        allVariables=[LpVariable("x_"+j,0,1,LpBinary) for j in featureTable]

        polyModel1 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model1+".pickle",'rb'))
        weights1 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model1+".weights",'rb'))
        polyModel2 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model2+".pickle",'rb'))
        weights2 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model2+".weights",'rb'))
        polyModel3 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model3+".pickle",'rb'))
        weights3 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model3+".weights",'rb'))

        weights =  weights3 # Go for the most destabilizing off target
        weights[0] = -1 # Don't include the offset...

        for threshold in [0]:
            print(threshold)
            
            # defines the problem
            prob = LpProblem("problem", LpMaximize) #########<<<<<<<<<<<<<<<<<<<<< MAXIMIZE
            addLinearConstraint_Forbiden(prob,allVariables)
            addLinearConstraintsPulp(22,prob,allVariables)
            addPairConstraintsPulp(22,prob,allVariables)



            ############# 
            #Affinity Constraints
            #
            #Affinity for target must be tighter than 
            #prob += LpAffineExpression((allVariables[k],weights1[k]) for k in range(len(allVariables))) <= -10 -polyModel1.intercept_[0]
            #prob += LpAffineExpression((allVariables[k],weights2[k]) for k in range(len(allVariables))) <= -10 -polyModel2.intercept_[0]
            #Affinity for offtarget must be less than 
            #prob += LpAffineExpression((allVariables[k],weights3[k]) for k in range(len(allVariables))) >= -8.5 -polyModel3.intercept_[0]
            #
            #############

            #############
            # Specificity Constraints
            #
            # (This determines the angle)  (-1 to 1) is the ratio of the two
            dualWeights = weights1-weights2
            dualIntercepts = polyModel1.intercept_[0]- polyModel2.intercept_[0]
            prob+= LpAffineExpression((allVariables[k],dualWeights[k]) for k in range(len(allVariables))) <= threshold+0.2 - dualIntercepts
            prob+= LpAffineExpression((allVariables[k],dualWeights[k]) for k in range(len(allVariables))) >= threshold-0.2 - dualIntercepts
            #
            #############

            #Problem to Solve For
            # Maximize the difference 
            dualWeights_affinity = weights3-weights1
            dualWeights_affinity[0] = -1
            prob += LpAffineExpression((allVariables[k],dualWeights_affinity[k]) for k in range(len(allVariables)))
            #prob += LpAffineExpression((allVariables[k],weights[k]) for k in range(len(allVariables)))


            topX = []
            for iteration in range(200):

                status = prob.solve(cplexSolver)
                LpStatus[status]

                if(LpStatus[status]=='Optimal'):                

                    allValues = [(idx,i,value(i)) for idx,i in enumerate(allVariables) if np.abs(value(i))>10**-6]
                    optSeq = "".join([str(allValues[i][1])[5:] for i in range(0,22)])

                    prob+= lpSum([allVariables[allValues[i][0]] for i in range(22)]) <= 21



                    #optSeq = receptor1+"_"+receptor2+str(iteration)

                    topX.append(optSeq)

                    #optSeqScore1 = polyModel1.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                    #optSeqScore2 = polyModel2.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                    #optSeqScore3 = polyModel3.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]

                    outFile.write(topX[-1])
                    outFile.write("\n")
                    outFile.flush()
                else:
                    outFile.write(LpStatus[status])
                    outFile.flush()
                    break


            

            

In [2]:
%%writefile /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1_edge_affinity.py


from pulp import *
import pickle
import pandas as pd
import sys
import numpy as np

aminoAcidIndex = 'ACEDGFIHKMLNQPSRTWVY'
myPosLabels=['1E','1F','1G','2A','2B','2C','2D','2E','2F','2G','3A','3B','3C','3D','3E','3F','3G','4A','4B','4C','4D','4E']


def kcalToKD(i):
    return np.e**(i/ ((1.9872041 * 10**-3)  * (298))) * 10**9

###############################
# Define ILP Functions to generate constraints
###############################

#Assuming 2 degree.
def addLinearConstraintsPulp(numAA,prob,allVariables,offset=1):
    
    allSingleAAConstraints = []
    
    for i in range(numAA):
        #blank = np.zeros(int((numAA*20)+((numAA*20)*((numAA*20)+1)/2)))
        prob+=lpSum(allVariables[i*20+j+offset] for j in range(20)) == 1
    


#Assuming 2 degree.
def addPairConstraintsPulp(numAA,prob,allVariables,offset=1):
    signs = [1,-1]        
    arrayOffset = numAA*20
    arrayDim = numAA*20
    #Expanded Vector1
    for i in range(numAA*20):
        ##################################################
        tempArray=[]
        #Expanded Vector2        
        for j in range(i,numAA*20):

            if(j%20==0 and j!=i): # For each set of 20, add another constraint
                
                tempArray.append((allVariables[i+offset],-1)) # AA constraint for single term
                prob += LpAffineExpression(each for each in tempArray) == 0
                tempArray=[]
                tempArray.append((allVariables[arrayOffset+offset],1))
            else:
                tempArray.append((allVariables[arrayOffset+offset],1))

        
            arrayOffset+=1
            
        tempArray.append((allVariables[i+offset],-1))   
        prob += LpAffineExpression(each for each in tempArray) == 0
        ##################################################
        
        
        #################################################
        # Second Set of constraints to assure second variable is stable
        #################################################
        
        
        tempArray2=[]
        #print(allVariables[i+offset], "=============", i)
        
        for k in range(i+1): # k is the number of times to add pair terms
            
            if(k>1 and k%20==0): #every 20, add another constraint that at most one pair can be selected
                
                # Assert that feature occurs as singleton
                tempArray2.append((allVariables[i+offset],-1))  
                prob += LpAffineExpression(each for each in tempArray2) == 0
                tempArray2=[]
            

            tempArray2.append((allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],1))
            #These calculations help get the index of the next pair term 
            
        
            #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],end=" ")
 
        
        #When complete, top it off and add the last set of constraints
        tempArray2.append((allVariables[i+offset],-1))
        #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],k,end=" ")
        prob += LpAffineExpression(each for each in tempArray2) == 0
        
        
def getPolyExpansionLabels(inputVector):
    
    myLabels = []
    

    myDegree = 2
    myNewVector=[]
    
    myLabels.append('Offset')
    for idx,each in enumerate(inputVector):
        myNewVector.append(np.sqrt(2)*each)
        myLabels.append(myPosLabels[idx//20]+"_"+aminoAcidIndex[idx%20])
    
    for i in range(len(inputVector)):
        for j in range(i,len(inputVector)):
            
            myLabels.append(myPosLabels[i//20]+"_"+aminoAcidIndex[i%20]+"__"+
            myPosLabels[j//20]+"_"+aminoAcidIndex[j%20])
    
    return myLabels

def getK(dim,iteration):
    mySum = 0
    for i in range(1,iteration+1):
        mySum+= (dim-i)
    return mySum

###############################
# Define Dummy Variable Encoding
###############################

def getAAVector(letter):
    vector = np.zeros(20);
    vector[aminoAcidIndex.index(letter)]=1
    return  vector

def encodeWithDummyVariables(sequence):
    newArray=[]
    for aa in sequence:
        newArray.append(getAAVector(aa))
    return np.array(newArray).ravel()

###############################
# Define Constraints on Counts
###############################
def addLinearConstraint_Forbiden(prob,allVariables,inputCounts,threshold,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bim):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
        
    for idx,each in enumerate(puma):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mixMut):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
    
    #Disallow low frequency observations
    for idx,each in enumerate(inputCounts):
        if(each)<threshold:
            allowedArray[idx]=0
    
    disallowed = (~allowedArray.astype(bool)).astype(int)
    
    tempArray=[]
    for idx,i in enumerate(disallowed):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == 0
    
    return allowedArray

#############################
# Define Library Constraints
#############################

bim = 'GRPEIWIAQELRRIGDEFNAYY'
puma = 'GQWAREIGAQLRRMADDLNAQY'
xMut = ['']*22
xMut[3] = 'DEHIKLMNQV'
xMut[6] ='DFHILNVY'
xMut[11]='AGIRTV'
xMut[13]='ADFGHILNPRSTVY' #C
xMut[14]='AG'
xMut[18]='ADHILNPTV'
xMut[19]='AEKT'
xMut[21]='ADGHNPRSTY' #C

mMut = ['']*22
mMut[2] = 'AGPRSTW'
mMut[4]='ADFGHLPRSVY' #C
mMut[5]='DEHQ'
mMut[6]='AITV'
mMut[7]='AGISTV'
mMut[11]='AEGIKRTV'
mMut[17]='ADFHILNPSTVY'
mMut[18]='DEHKNQ'

fMut = ['']*22
fMut[3] = 'AEIKLPQTV'
fMut[7]='ADGSY' #C
fMut[9]='DFGHILNRSVY'# C
fMut[13]='AFGILPRSTV' #C
fMut[16]='DEHIKLMNQV'
fMut[17]='ADFHILNPSTVY'
fMut[21]='AFILPSTV'

mixMut = []
for i in range(22):
    mixMut.append(set(xMut[i]).union(set(mMut[i])).union(set(fMut[i])))

libVariables = [LpVariable("i_"+j,0,22,LpInteger) for j in ['bx','bm','bf','px','pm','pf']] 
libVariablesBinary = [LpVariable("b_"+j,0,22,LpBinary) for j in ['bx','bm','bf','px','pm','pf']] 

###############################
# If Designing in the input library space use the following functions
###############################

def addLinearConstraintSingleLib(prob,allVariables,bgLib,mutAllowed,libVar,libVarBin,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bgLib):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mutAllowed):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
            
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<10:
            allowedArray[idx]=0
    
    tempArray=[]
    for idx,i in enumerate(allowedArray):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == libVar
    prob += libVar >= libVarBin*22 # libVarBin is 1 if libVar is >= 22
    #Because of the other constraints - this is just a variable that assigns boolean == 22
    
    return tempArray
    
def addLinearConstraintMultipleLib(prob,allVariables,offset=1):
    addLinearConstraintSingleLib(prob,allVariables,bim,xMut,libVariables[0],libVariablesBinary[0],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,mMut,libVariables[1],libVariablesBinary[1],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,fMut,libVariables[2],libVariablesBinary[2],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,xMut,libVariables[3],libVariablesBinary[3],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,mMut,libVariables[4],libVariablesBinary[4],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,fMut,libVariables[5],libVariablesBinary[5],offset)
def addLinearConstraint_library(prob,allVariables,offset=1):
    prob+= lpSum(each for each in libVariablesBinary) >= 1
    addLinearConstraintMultipleLib(prob,allVariables,offset)
    #print(libVariables)

    
    
    
#############################
# Start loading model specific reqs
#############################

allData,allDataName = pickle.load(open('/home/vxue/data/sort_specificity/ncv_y/allData.pickle','rb'))

modelIndex = [allDataName.index(i) for i in ['all_x','all_s',
                                             'all_m','all_n',
                                             'all_f','all_t',
                                             'all_z','all_c']]

myIndex = [row+'_'+letter for row in myPosLabels for letter in aminoAcidIndex]

#############################
# Count the number of sequences
#############################
myCounts = dict()
for each,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myCounts[name]=posAA_toCount

    
#############################
# Count the number of sequences that have an affinity less than -10..5
#############################
myBindingCounts = dict()
for each_full,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    #Count only the ones which have affinity less than ~50 nM
    each = each_full[each_full.yValue < -10.5].copy().reset_index()
    
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myBindingCounts[name]=posAA_toCount
    
    
    
designsTargets = [  #Do the pairs by the ones that have the best cross validated performance
    ('fn','x'),
    ('nx','f'),
    ('xf','n')]

slurmInput =int(sys.argv[1])    


cplexSolver = solvers.CPLEX_CMD("/scratch/users/mit_keating/Applications/CPLEX_Studio1271/cplex/bin/x86-64_linux/cplex")

for receptor1,receptor2 in [designsTargets[slurmInput]]:
    
    with open("/home/vxue/data/sort_specificity/design_y/specificity/trial9/"+receptor1+"-"+receptor2+"_poly.csv",'w') as outFile:
        model1 = "all_"+receptor1[0]
        model2 = "all_"+receptor1[1]
        model3 = "all_"+receptor2


        print(model1,model2,model3)
        #Seq counts are set to the min of the three targets
        aaCounts1  = [myCounts[model1][i] for i in myIndex]
        aaCounts2  = [myCounts[model2][i] for i in myIndex]
        aaCounts3  = [myCounts[model3][i] for i in myIndex]
        aaCounts = min(aaCounts1,aaCounts2,aaCounts3)

        #Binding counts are set to the min of the two targets
        bindingCounts1 = [myBindingCounts[model1][i] for i in myIndex]
        bindingCounts2 = [myBindingCounts[model2][i] for i in myIndex]
        bindingCounts = min(bindingCounts1,bindingCounts2)
        
        
        
        featureTable = getPolyExpansionLabels(np.ones(440))
        allVariables=[LpVariable("x_"+j,0,1,LpBinary) for j in featureTable]

        polyModel1 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model1+".pickle",'rb'))
        weights1 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model1+".weights",'rb'))
        polyModel2 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model2+".pickle",'rb'))
        weights2 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model2+".weights",'rb'))
        polyModel3 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model3+".pickle",'rb'))
        weights3 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model3+".weights",'rb'))

        weights =  weights3 # Go for the most destabilizing off target
        weights[0] = -1 # Don't include the offset...

        for threshold in [0]:
            print(threshold)
            
            # defines the problem
            prob = LpProblem("problem", LpMaximize) #########<<<<<<<<<<<<<<<<<<<<< MAXIMIZE
            addLinearConstraint_Forbiden(prob,allVariables,aaCounts,25)
            addLinearConstraint_Forbiden(prob,allVariables,bindingCounts,1)
            addLinearConstraintsPulp(22,prob,allVariables)
            addPairConstraintsPulp(22,prob,allVariables)



            ############# 
            #Affinity Constraints
            #
            #Affinity for target must be tighter than 
            prob += LpAffineExpression((allVariables[k],weights1[k]) for k in range(len(allVariables))) <= -10 -polyModel1.intercept_[0]
            prob += LpAffineExpression((allVariables[k],weights2[k]) for k in range(len(allVariables))) <= -10 -polyModel2.intercept_[0]
            #Affinity for offtarget must be less than 
            #prob += LpAffineExpression((allVariables[k],weights3[k]) for k in range(len(allVariables))) >= -8.5 -polyModel3.intercept_[0]
            #
            #############

            #############
            # Specificity Constraints
            #
            # (This determines the angle)  (-1 to 1) is the ratio of the two
            dualWeights = weights1-weights2
            dualIntercepts = polyModel1.intercept_[0]- polyModel2.intercept_[0]
            prob+= LpAffineExpression((allVariables[k],dualWeights[k]) for k in range(len(allVariables))) <= threshold+0.2 - dualIntercepts
            prob+= LpAffineExpression((allVariables[k],dualWeights[k]) for k in range(len(allVariables))) >= threshold-0.2 - dualIntercepts
            #
            #############

            #Problem to Solve For
            # Maximize the difference 
            dualWeights_affinity = weights3-weights1
            dualWeights_affinity[0] = -1
            prob += LpAffineExpression((allVariables[k],dualWeights_affinity[k]) for k in range(len(allVariables)))
            #prob += LpAffineExpression((allVariables[k],weights[k]) for k in range(len(allVariables)))


            topX = []
            for iteration in range(200):

                status = prob.solve(cplexSolver)
                LpStatus[status]

                if(LpStatus[status]=='Optimal'):                

                    allValues = [(idx,i,value(i)) for idx,i in enumerate(allVariables) if np.abs(value(i))>10**-6]
                    optSeq = "".join([str(allValues[i][1])[5:] for i in range(0,22)])

                    prob+= lpSum([allVariables[allValues[i][0]] for i in range(22)]) <= 21



                    #optSeq = receptor1+"_"+receptor2+str(iteration)

                    topX.append(optSeq)

                    #optSeqScore1 = polyModel1.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                    #optSeqScore2 = polyModel2.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                    #optSeqScore3 = polyModel3.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]

                    outFile.write(topX[-1])
                    outFile.write("\n")
                    outFile.flush()
                else:
                    outFile.write(LpStatus[status])
                    outFile.flush()
                    break


            

            

Overwriting /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1_edge_affinity.py


In [8]:
%%writefile /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1_maximize_affinity_targets.py


from pulp import *
import pickle
import pandas as pd
import sys
import numpy as np

aminoAcidIndex = 'ACEDGFIHKMLNQPSRTWVY'
myPosLabels=['1E','1F','1G','2A','2B','2C','2D','2E','2F','2G','3A','3B','3C','3D','3E','3F','3G','4A','4B','4C','4D','4E']


def kcalToKD(i):
    return np.e**(i/ ((1.9872041 * 10**-3)  * (298))) * 10**9

###############################
# Define ILP Functions to generate constraints
###############################

#Assuming 2 degree.
def addLinearConstraintsPulp(numAA,prob,allVariables,offset=1):
    
    allSingleAAConstraints = []
    
    for i in range(numAA):
        #blank = np.zeros(int((numAA*20)+((numAA*20)*((numAA*20)+1)/2)))
        prob+=lpSum(allVariables[i*20+j+offset] for j in range(20)) == 1
    


#Assuming 2 degree.
def addPairConstraintsPulp(numAA,prob,allVariables,offset=1):
    signs = [1,-1]        
    arrayOffset = numAA*20
    arrayDim = numAA*20
    #Expanded Vector1
    for i in range(numAA*20):
        ##################################################
        tempArray=[]
        #Expanded Vector2        
        for j in range(i,numAA*20):

            if(j%20==0 and j!=i): # For each set of 20, add another constraint
                
                tempArray.append((allVariables[i+offset],-1)) # AA constraint for single term
                prob += LpAffineExpression(each for each in tempArray) == 0
                tempArray=[]
                tempArray.append((allVariables[arrayOffset+offset],1))
            else:
                tempArray.append((allVariables[arrayOffset+offset],1))

        
            arrayOffset+=1
            
        tempArray.append((allVariables[i+offset],-1))   
        prob += LpAffineExpression(each for each in tempArray) == 0
        ##################################################
        
        
        #################################################
        # Second Set of constraints to assure second variable is stable
        #################################################
        
        
        tempArray2=[]
        #print(allVariables[i+offset], "=============", i)
        
        for k in range(i+1): # k is the number of times to add pair terms
            
            if(k>1 and k%20==0): #every 20, add another constraint that at most one pair can be selected
                
                # Assert that feature occurs as singleton
                tempArray2.append((allVariables[i+offset],-1))  
                prob += LpAffineExpression(each for each in tempArray2) == 0
                tempArray2=[]
            

            tempArray2.append((allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],1))
            #These calculations help get the index of the next pair term 
            
        
            #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],end=" ")
 
        
        #When complete, top it off and add the last set of constraints
        tempArray2.append((allVariables[i+offset],-1))
        #print(allVariables[(arrayDim+offset+i+ getK(arrayDim,k))],k,end=" ")
        prob += LpAffineExpression(each for each in tempArray2) == 0
        
        
def getPolyExpansionLabels(inputVector):
    
    myLabels = []
    

    myDegree = 2
    myNewVector=[]
    
    myLabels.append('Offset')
    for idx,each in enumerate(inputVector):
        myNewVector.append(np.sqrt(2)*each)
        myLabels.append(myPosLabels[idx//20]+"_"+aminoAcidIndex[idx%20])
    
    for i in range(len(inputVector)):
        for j in range(i,len(inputVector)):
            
            myLabels.append(myPosLabels[i//20]+"_"+aminoAcidIndex[i%20]+"__"+
            myPosLabels[j//20]+"_"+aminoAcidIndex[j%20])
    
    return myLabels

def getK(dim,iteration):
    mySum = 0
    for i in range(1,iteration+1):
        mySum+= (dim-i)
    return mySum

###############################
# Define Dummy Variable Encoding
###############################

def getAAVector(letter):
    vector = np.zeros(20);
    vector[aminoAcidIndex.index(letter)]=1
    return  vector

def encodeWithDummyVariables(sequence):
    newArray=[]
    for aa in sequence:
        newArray.append(getAAVector(aa))
    return np.array(newArray).ravel()

###############################
# Define Constraints on Counts
###############################
def addLinearConstraint_Forbiden(prob,allVariables,inputCounts,threshold,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bim):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
        
    for idx,each in enumerate(puma):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mixMut):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
    
    #Disallow low frequency observations
    for idx,each in enumerate(inputCounts):
        if(each)<threshold:
            allowedArray[idx]=0
    
    disallowed = (~allowedArray.astype(bool)).astype(int)
    
    tempArray=[]
    for idx,i in enumerate(disallowed):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == 0
    
    return allowedArray

#############################
# Define Library Constraints
#############################

bim = 'GRPEIWIAQELRRIGDEFNAYY'
puma = 'GQWAREIGAQLRRMADDLNAQY'
xMut = ['']*22
xMut[3] = 'DEHIKLMNQV'
xMut[6] ='DFHILNVY'
xMut[11]='AGIRTV'
xMut[13]='ADFGHILNPRSTVY' #C
xMut[14]='AG'
xMut[18]='ADHILNPTV'
xMut[19]='AEKT'
xMut[21]='ADGHNPRSTY' #C

mMut = ['']*22
mMut[2] = 'AGPRSTW'
mMut[4]='ADFGHLPRSVY' #C
mMut[5]='DEHQ'
mMut[6]='AITV'
mMut[7]='AGISTV'
mMut[11]='AEGIKRTV'
mMut[17]='ADFHILNPSTVY'
mMut[18]='DEHKNQ'

fMut = ['']*22
fMut[3] = 'AEIKLPQTV'
fMut[7]='ADGSY' #C
fMut[9]='DFGHILNRSVY'# C
fMut[13]='AFGILPRSTV' #C
fMut[16]='DEHIKLMNQV'
fMut[17]='ADFHILNPSTVY'
fMut[21]='AFILPSTV'

mixMut = []
for i in range(22):
    mixMut.append(set(xMut[i]).union(set(mMut[i])).union(set(fMut[i])))

libVariables = [LpVariable("i_"+j,0,22,LpInteger) for j in ['bx','bm','bf','px','pm','pf']] 
libVariablesBinary = [LpVariable("b_"+j,0,22,LpBinary) for j in ['bx','bm','bf','px','pm','pf']] 

###############################
# If Designing in the input library space use the following functions
###############################

def addLinearConstraintSingleLib(prob,allVariables,bgLib,mutAllowed,libVar,libVarBin,offset=1):
    allowedArray = np.zeros(20*22)
    #Permit Native Seqs
    for idx,each in enumerate(bgLib):
        allowedArray[20*idx+aminoAcidIndex.index(each)] = 1
    
    for site,mutations in enumerate(mutAllowed):
        for each in mutations:
            allowedArray[site*20+aminoAcidIndex.index(each)]=1
            
    #Disallow low frequency observations
    for idx,each in enumerate(aaCounts):
        if(each)<10:
            allowedArray[idx]=0
    
    tempArray=[]
    for idx,i in enumerate(allowedArray):
        if(i==1):
            tempArray.append((allVariables[idx+offset],1))   

    #print(tempArray)
    prob += LpAffineExpression(each for each in tempArray) == libVar
    prob += libVar >= libVarBin*22 # libVarBin is 1 if libVar is >= 22
    #Because of the other constraints - this is just a variable that assigns boolean == 22
    
    return tempArray
    
def addLinearConstraintMultipleLib(prob,allVariables,offset=1):
    addLinearConstraintSingleLib(prob,allVariables,bim,xMut,libVariables[0],libVariablesBinary[0],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,mMut,libVariables[1],libVariablesBinary[1],offset)
    addLinearConstraintSingleLib(prob,allVariables,bim,fMut,libVariables[2],libVariablesBinary[2],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,xMut,libVariables[3],libVariablesBinary[3],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,mMut,libVariables[4],libVariablesBinary[4],offset)
    addLinearConstraintSingleLib(prob,allVariables,puma,fMut,libVariables[5],libVariablesBinary[5],offset)
def addLinearConstraint_library(prob,allVariables,offset=1):
    prob+= lpSum(each for each in libVariablesBinary) >= 1
    addLinearConstraintMultipleLib(prob,allVariables,offset)
    #print(libVariables)

    
    
    
#############################
# Start loading model specific reqs
#############################

allData,allDataName = pickle.load(open('/home/vxue/data/sort_specificity/ncv_y/allData.pickle','rb'))

modelIndex = [allDataName.index(i) for i in ['all_x','all_s',
                                             'all_m','all_n',
                                             'all_f','all_t',
                                             'all_z','all_c']]

myIndex = [row+'_'+letter for row in myPosLabels for letter in aminoAcidIndex]

#############################
# Count the number of sequences
#############################
myCounts = dict()
for each,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myCounts[name]=posAA_toCount

    
#############################
# Count the number of sequences that have an affinity less than -10..5
#############################
myBindingCounts = dict()
for each_full,name in zip([allData[i] for i in modelIndex],[allDataName[i] for i in modelIndex]):
    #Count only the ones which have affinity less than ~50 nM
    each = each_full[each_full.yValue < -10.5].copy().reset_index()
    
    each['aaEncoding'] = each.apply(lambda x: list(encodeWithDummyVariables(x.twentytwo)),axis=1)
    myMatrix = np.array([np.array(i) for i in each.aaEncoding])
    counts = myMatrix.sum(axis=0)
    posAA_toCount = dict()
    for i,count in zip(myIndex,counts):
        posAA_toCount[i] = int(count)
    myBindingCounts[name]=posAA_toCount
    
    
    
designsTargets = [  #Do the pairs by the ones that have the best cross validated performance
    ('fn','x'),
    ('nx','f'),
    ('xf','n')]

slurmInput =int(sys.argv[1])    


cplexSolver = solvers.CPLEX_CMD("/scratch/users/mit_keating/Applications/CPLEX_Studio1271/cplex/bin/x86-64_linux/cplex")

for receptor1,receptor2 in [designsTargets[slurmInput]]:
    
    with open("/home/vxue/data/sort_specificity/design_y/specificity/trial10/"+receptor1+"-"+receptor2+"_poly.csv",'w') as outFile:
        model1 = "all_"+receptor1[0]
        model2 = "all_"+receptor1[1]
        model3 = "all_"+receptor2


        print(model1,model2,model3)
        #Seq counts are set to the min of the three targets
        aaCounts1  = [myCounts[model1][i] for i in myIndex]
        aaCounts2  = [myCounts[model2][i] for i in myIndex]
        aaCounts3  = [myCounts[model3][i] for i in myIndex]
        aaCounts = min(aaCounts1,aaCounts2,aaCounts3)

        #Binding counts are set to the min of the two targets
        bindingCounts1 = [myBindingCounts[model1][i] for i in myIndex]
        bindingCounts2 = [myBindingCounts[model2][i] for i in myIndex]
        bindingCounts = min(bindingCounts1,bindingCounts2)
        
        
        
        featureTable = getPolyExpansionLabels(np.ones(440))
        allVariables=[LpVariable("x_"+j,0,1,LpBinary) for j in featureTable]

        polyModel1 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model1+".pickle",'rb'))
        weights1 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model1+".weights",'rb'))
        polyModel2 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model2+".pickle",'rb'))
        weights2 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model2+".weights",'rb'))
        polyModel3 = pickle.load(open("/home/vxue/data/sort_specificity/rmse_doubleSet/ncv_y/modelpoly"+model3+".pickle",'rb'))
        weights3 = pickle.load(open("/home/vxue/data/sort_specificity/design_y/weights/ncv_y.modelpoly" +model3+".weights",'rb'))

        weights =  weights1 # Go for the most stabilizing target
        weights[0] = 1 # Don't include the offset

        for threshold in [0]:
            print(threshold)
            
            # defines the problem
            prob = LpProblem("problem", LpMinimize)
            addLinearConstraint_Forbiden(prob,allVariables,aaCounts,25)
            addLinearConstraint_Forbiden(prob,allVariables,bindingCounts,1)
            addLinearConstraintsPulp(22,prob,allVariables)
            addPairConstraintsPulp(22,prob,allVariables)



            ############# 
            #Affinity Constraints
            #
            #Affinity for target must be tighter than 
            #prob += LpAffineExpression((allVariables[k],weights1[k]) for k in range(len(allVariables))) <= -10 -polyModel1.intercept_[0]
            #prob += LpAffineExpression((allVariables[k],weights2[k]) for k in range(len(allVariables))) <= -10 -polyModel2.intercept_[0]
            #Affinity for offtarget must be less than 
            prob += LpAffineExpression((allVariables[k],weights3[k]) for k in range(len(allVariables))) >= -9 -polyModel3.intercept_[0]
            #
            #############

            #############
            # Specificity Constraints
            #
            # (This determines the angle)  (-1 to 1) is the ratio of the two
            dualWeights = weights1-weights2
            dualIntercepts = polyModel1.intercept_[0]- polyModel2.intercept_[0]
            prob+= LpAffineExpression((allVariables[k],dualWeights[k]) for k in range(len(allVariables))) <= threshold+0.2 - dualIntercepts
            prob+= LpAffineExpression((allVariables[k],dualWeights[k]) for k in range(len(allVariables))) >= threshold-0.2 - dualIntercepts
            #
            #############

            #Problem to Solve For
            # Maximize the difference 
            #dualWeights_affinity = weights3-weights1
            #dualWeights_affinity[0] = -1
            #prob += LpAffineExpression((allVariables[k],dualWeights_affinity[k]) for k in range(len(allVariables)))
            prob += LpAffineExpression((allVariables[k],weights[k]) for k in range(len(allVariables)))


            topX = []
            for iteration in range(200):

                status = prob.solve(cplexSolver)
                LpStatus[status]

                if(LpStatus[status]=='Optimal'):                

                    allValues = [(idx,i,value(i)) for idx,i in enumerate(allVariables) if np.abs(value(i))>10**-6]
                    optSeq = "".join([str(allValues[i][1])[5:] for i in range(0,22)])

                    prob+= lpSum([allVariables[allValues[i][0]] for i in range(22)]) <= 21



                    #optSeq = receptor1+"_"+receptor2+str(iteration)

                    topX.append(optSeq)

                    #optSeqScore1 = polyModel1.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                    #optSeqScore2 = polyModel2.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]
                    #optSeqScore3 = polyModel3.predict(encodeWithDummyVariables(optSeq).reshape(1,-1))[0]

                    outFile.write(topX[-1])
                    outFile.write("\n")
                    outFile.flush()
                else:
                    outFile.write(LpStatus[status])
                    outFile.flush()
                    break


            

            

Writing /home/vxue/data/sort_specificity/design_y/scripts/poly_specificty_2v1_maximize_affinity_targets.py
